# 1. Baseline

## Libraries

In [1]:
"""
!pip install torch
!pip install easydict
!pip install torch
!pip install torchvision
!pip install opencv-python
"""

'\n!pip install torch\n!pip install easydict\n!pip install torch\n!pip install torchvision\n!pip install opencv-python\n'

In [2]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the CUDA version
    cuda_version = torch.version.cuda
    print(f"CUDA version: {cuda_version}")
else:
    print("CUDA is not available. Install CUDA and try again.")

CUDA is not available. Install CUDA and try again.


In [3]:

import os
from pathlib import Path
from tqdm import tqdm #eğitim kısmını görselleştirmede kullanılır
from easydict import EasyDict as edict

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
from torchvision import transforms as trans

from data.ms1m import get_train_loader
from data.lfw import LFW

from backbone.arcfacenet import SEResNet_IR
from margin.ArcMarginProduct import ArcMarginProduct

from util.utils import save_checkpoint, test


## Configuration

In [4]:
conf = edict()
conf.train_root = "./dataset/MS1M"
conf.lfw_test_root = "./dataset/lfw_aligned_112"
conf.lfw_file_list = "./dataset/lfw_pair.txt"

conf.mode = "se_ir" #model mode'u. backbone içindeki arcnet içinde bulunan ir modelini kullancaz. ir resnet temelli, se_ir resnet bloklarının yanında se blokları da barındırıyor
conf.depth = 50 #50, 100 ya da 152 seçilebilir. arcfacenet.py dosyasının içinde görülüyor. Daha doğru sonuç için arttırılabilir 100'e fln
conf.margin_type ="ArcFace"
conf.feature_dim = 512
conf.scale_size = 32.0
conf.batch_size = 48 #16'nın katları olarak seçilir genelde, 96 ORİJİNALDE
conf.lr = 0.005 #learning rate, batchsize 16 yaparsak lr daha düşük olmalı , 0.01 ORİJİNALDE
conf.milestones = [8,10,12] #lr burdaki epochlarda düşecek
conf.total_epoch = 14

conf.save_folder = "./saved"
conf.save_dir = os.path.join(conf.save_folder, conf.mode +"_" + str(conf.depth)) #./saved/se_ir_50
conf.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
conf.num_workers = 4 #gpuda dataları yüklemek için kaç worker kullanmak istediğimizi giriyoruz
conf.pin_memory = True #gpu ile eğitim sırasında daha hızlı sonuçlar almamızı sağlar

In [5]:
os.makedirs(conf.save_dir, exist_ok = True)

## Data Loader

In [6]:
transform = trans.Compose([ #birden fazla transformasyon varsa bunları birleştiriyo
    trans.ToTensor(), #değişkenleri tensore dönüştürür, range [0,255] iken [0.0,1.0] olarak dönüştürüyor
    trans.Normalize(mean = (0.5,0.5,0.5), std = (0.5,0.5,0.5)) #r,g,b . Daha dayanıklı modeller olşuyor normalizasyonla, genelde 0.5 kullanılır. Örneğin fotoda çok parlak alanlar fln varsa
])

trainloader , class_num = get_train_loader(conf) #ms1m.py dosyası içinde bu

In [7]:
print("number of id :", class_num) #200 farklı kişinin resimleri var

number of id : 200


In [8]:
print(trainloader.dataset)

Dataset ImageFolder
    Number of datapoints: 29148
    Root location: ./dataset/MS1M
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )


In [9]:
#üstteki işlemleri test kümesi için de yapıyoruz
lfwdataset = LFW(conf.lfw_test_root, conf.lfw_file_list, transform = transform)
#yukarda yaptığımız şey aslında şu: conf.lfw_test_root adresinde bulunan conf.lfw_file_list label'lı resimlere transform transformasyonunu uygula demek

#yukardan gelen verileri yükleyebilmek için loader'a ihtiyacımız var
lfwloader = torch.utils.data.DataLoader(lfwdataset, batch_size = 128, num_workers = conf.num_workers)

# Model

In [18]:
print(conf.device)

cpu


In [11]:
net = SEResNet_IR(conf.depth, feature_dim = conf.feature_dim, mode = conf.mode).to(conf.device) #model
margin = ArcMarginProduct(conf.feature_dim, class_num).to(conf.device)


In [12]:
print(net)

SEResNet_IR(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (body): Sequential(
    (0): BottleNeck_IR_SE(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, moment

In [13]:
criterion = nn.CrossEntropyLoss()

In [14]:
optimizer = optim.SGD([
    {"params": net.parameters(), "weight_decay":5e-4}, #wight_decay overfittingi önlemek için kullanılan dropout gibi bir yöntem
    {"params": margin.parameters(), "weight_decay":5e-4}
], lr = conf.lr, momentum = 0.9, nesterov = True)

In [15]:
print(optimizer, flush  = True)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.005
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005

Parameter Group 1
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.005
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005
)


In [16]:
def schedule_lr():
    for params in optimizer.param_groups:
        params["lr"]/=10
    print(optimizer)

## Train

In [ ]:
best_acc = 0 
for epoch in range(1, conf.total_epoch+1):
    net.train()

    print("epoch {}/{}".format(epoch, conf.total_epoch), flush = True)

    if epoch == conf.milestones[0]: #8.epoch
        schedule_lr()
    if epoch == conf.milestones[1]: #10.epoch
        schedule_lr()
    if epoch == conf.milestones[2]: #12.epoch
        schedule_lr()
#yukarıyı tekrar çalıştırdığında lr başlangıç değeri olan değerden değil de en so düştüğü değerden itibaren tekrar düşer, o yüzden modeli ve optimizer'ı tekrar yüklemen gerekir
    for data in tqdm(trainloader): #tqdm görsellik sağlıyormuş???
        img, label = data[0].to(conf.device), data[1].to(conf.device)
        optimizer.zero_grad() #optimizer'ın gradyanlarını sıfırlıyoruz. Backward aşamasında gelen errorlerin üstüste binmemesi için sıfırlıyoruz

        logits = net(img) #resimleri modele gönderip çıktı alıyoruz, (96,512)lik bir vektörümüz oldu
        output = margin(logits,label)
        total_loss = criterion(output, label)
        total_loss.backward()
        optimizer.step()

    #test
    bet.eval()
    lfw_acc = test(conf, net,lfwdataset, lfwloader)
    print("\nLFW: {:.4f} | train_loss: {:.4f}\n".format(lfw_acc, total_loss.item()))
    is_best = lfw_acc > best_acc
    best_acc = max(lfw_acc, best_acc)
    save_checkpoint({
        "epoch":epoch,
        "net_state_dict": net.state_dict(),
        "margin_state_dict":margin.state_dict(),
        "best_acc": best_acc
    }, is_best,checkpoint = conf.save_dir)

In [ ]:
"""
SOTA Ç The state of the art (CASIA)

1. MS1M datasetinin tamaminin indirilmesi
2. conf.mode = "ir"
3. conf.depth = "100"
4. conf.total_epoch = 20
5. conf.milestones = [12,16,18]


# 

"""